<a href="https://colab.research.google.com/github/ferhat00/LLM/blob/main/Satellite_Imagery_Cushing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install astropy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.7/738.7 kB 43.8 MB/s eta 0:00:00


In [4]:
!pip install shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.1 MB/s eta 0:00:00


In [17]:
!pip install pytz

In [20]:
import numpy as np
import cv2
from datetime import datetime
import pytz # Import the pytz module
from astropy.coordinates import get_sun, EarthLocation, AltAz
import astropy.units as u
from shapely.geometry import Polygon
import pandas as pd
from astropy.time import Time # Import the Time class

class OilTankAnalyzer:
    def __init__(self):
        # Cushing, Oklahoma coordinates
        self.latitude = 35.9828  # N
        self.longitude = -96.7534  # W
        self.location = EarthLocation(lat=self.latitude*u.deg,
                                    lon=self.longitude*u.deg,
                                    height=300*u.m)
        self.timezone = pytz.timezone('US/Central') # Define the timezone, here set to 'US/Central' for Cushing, Oklahoma

    def calculate_sun_position(self, timestamp):
        """Calculate sun's altitude and azimuth for shadow analysis."""
        # Convert local time to UTC for astronomical calculations
        if timestamp.tzinfo is None:
            local_time = self.timezone.localize(timestamp)
        else:
            local_time = timestamp.astimezone(self.timezone)

        utc_time = local_time.astimezone(pytz.UTC)
        # Convert datetime to astropy Time object
        utc_time = Time(utc_time, scale='utc') # Convert to astropy Time object with scale='utc'
        altaz = AltAz(location=self.location, obstime=utc_time)
        sun = get_sun(utc_time)
        sun_altaz = sun.transform_to(altaz)
        return sun_altaz.alt.deg, sun_altaz.az.deg

    def detect_tanks(self, image):
        """
        Detect circular tanks using Hough Circle Transform
        Returns: List of (x, y, radius) for each detected tank
        """
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (9, 9), 2)

        # Detect circles using Hough Circle Transform
        circles = cv2.HoughCircles(
            blurred,
            cv2.HOUGH_GRADIENT,
            dp=1,
            minDist=50,
            param1=50,
            param2=30,
            minRadius=20,
            maxRadius=100
        )

        if circles is not None:
            return np.round(circles[0, :]).astype(int)
        return []

    def analyze_shadow(self, image, tank_info, sun_alt, sun_az):
        """
        Analyze shadow pattern to estimate tank fill level
        Args:
            image: Satellite image
            tank_info: (x, y, radius) of tank
            sun_alt: Sun altitude in degrees
            sun_az: Sun azimuth in degrees
        Returns:
            Estimated fill percentage
        """
        x, y, r = tank_info

        # Create mask for shadow detection
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.circle(mask, (x, y), r, 255, -1)

        # Convert to HSV for better shadow detection
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        shadow_mask = cv2.inRange(hsv,
                                np.array([0, 0, 0]),
                                np.array([180, 255, 70]))

        # Combine tank mask with shadow detection
        shadow_region = cv2.bitwise_and(shadow_mask, mask)

        # Calculate expected shadow direction based on sun position
        shadow_angle = sun_az + 180  # Shadow falls opposite to sun
        shadow_length = r * np.tan(np.radians(90 - sun_alt))

        # Create expected shadow polygon
        shadow_end_x = x + shadow_length * np.cos(np.radians(shadow_angle))
        shadow_end_y = y + shadow_length * np.sin(np.radians(shadow_angle))

        # Compare actual shadow with expected shadow
        expected_shadow = Polygon([
            (x, y),
            (shadow_end_x, shadow_end_y),
            (shadow_end_x + r, shadow_end_y),
            (x + r, y)
        ])

        actual_shadow = np.where(shadow_region > 0)
        actual_shadow_area = len(actual_shadow[0])

        # Calculate fill level based on shadow pattern
        max_shadow_area = np.pi * r * shadow_length
        shadow_ratio = actual_shadow_area / max_shadow_area

        # Adjust for floating roof tanks
        fill_percentage = 100 * (1 - shadow_ratio)

        return fill_percentage

    def estimate_volume(self, tank_radius, fill_percentage):
        """
        Estimate stored volume based on tank dimensions and fill level
        Args:
            tank_radius: Radius in pixels (needs conversion to meters)
            fill_percentage: Estimated fill percentage
        Returns:
            Estimated volume in barrels
        """
        # Convert pixel radius to meters (approximate conversion based on typical tank sizes)
        pixels_per_meter = 0.5  # This needs calibration based on image resolution
        radius_meters = tank_radius * pixels_per_meter

        # Standard tank height (most Cushing tanks are ~40-50 feet tall)
        height_meters = 14  # approximately 45 feet

        # Calculate total tank volume in cubic meters
        total_volume = np.pi * radius_meters**2 * height_meters

        # Convert to barrels (1 cubic meter = 6.2898 barrels)
        total_barrels = total_volume * 6.2898

        # Apply fill percentage
        actual_barrels = total_barrels * (fill_percentage / 100)

        return actual_barrels

    def analyze_image(self, image_path, timestamp):
        """
        Main function to analyze satellite image and estimate storage volumes
        Args:
            image_path: Path to satellite image
            timestamp: DateTime object of image capture
        Returns:
            DataFrame with tank locations and estimated volumes
        """
        # Load and preprocess image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Could not load image")

        # Get sun position
        sun_alt, sun_az = self.calculate_sun_position(timestamp)

        # Detect tanks
        tanks = self.detect_tanks(image)

        results = []
        for tank in tanks:
            x, y, r = tank
            fill_percentage = self.analyze_shadow(image, tank, sun_alt, sun_az)
            volume = self.estimate_volume(r, fill_percentage)

            results.append({
                'x': x,
                'y': y,
                'radius': r,
                'fill_percentage': fill_percentage,
                'estimated_volume': volume
            })

        return pd.DataFrame(results)

def main():
    analyzer = OilTankAnalyzer()
    timestamp = datetime.now()  # Replace with actual image timestamp
    results = analyzer.analyze_image('/content/gas.jpg', timestamp)
    print(f"Total estimated storage: {results['estimated_volume'].sum():,.0f} barrels")
    print(f"Fill percentage: {results['fill_percentage'].mean():,.0f} %")
    return results

if __name__ == "__main__":
    main()

Total estimated storage: 7,599,171 barrels
Fill percentage: 99 %


In [25]:
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
import pytz
from astropy.coordinates import get_sun, EarthLocation, AltAz
import astropy.units as u
from astropy.time import Time
import pandas as pd
from torchvision import transforms, models
from sklearn.preprocessing import StandardScaler

class TankDetectionNet(nn.Module):
    def __init__(self):
        super(TankDetectionNet, self).__init__()
        # Use ResNet50 as backbone, pretrained on ImageNet
        self.backbone = models.resnet50(pretrained=True)

        # Modify final layers for tank detection
        num_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()

        # Custom detection heads
        self.location_head = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 4)  # x, y, width, height
        )

        self.confidence_head = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        features = self.backbone(x)
        locations = self.location_head(features)
        confidence = self.confidence_head(features)
        return locations, confidence

class ShadowAnalysisNet(nn.Module):
    def __init__(self):
        super(ShadowAnalysisNet, self).__init__()
        # U-Net style architecture for shadow segmentation
        self.encoder = models.resnet34(pretrained=True)
        self.encoder.fc = nn.Identity()

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2),
            nn.Sigmoid()
        )

    def forward(self, x):
        features = self.encoder(x)
        features = features.view(-1, 512, 7, 7)  # Reshape for decoder
        shadow_mask = self.decoder(features)
        return shadow_mask

class VolumeEstimationNet(nn.Module):
    def __init__(self):
        super(VolumeEstimationNet, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(7, 128),  # Input: shadow metrics + sun position + tank dimensions
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)    # Output: fill percentage
        )

    def forward(self, x):
        return self.network(x)

class MLTankAnalyzer:
    def __init__(self, model_weights_path=None):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Initialize models
        self.tank_detector = TankDetectionNet().to(self.device)
        self.shadow_analyzer = ShadowAnalysisNet().to(self.device)
        self.volume_estimator = VolumeEstimationNet().to(self.device)

        # Load pre-trained weights if provided
        if model_weights_path:
            self.load_weights(model_weights_path)

        # Cushing coordinates
        self.latitude = 35.9828  # N
        self.longitude = -96.7534  # W
        self.location = EarthLocation(lat=self.latitude*u.deg,
                                    lon=self.longitude*u.deg,
                                    height=300*u.m)

        # Image preprocessing
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
        ])

        self.scaler = StandardScaler()

    def load_weights(self, weights_path):
        """Load pre-trained weights for all models"""
        weights = torch.load(weights_path, map_location=self.device)
        self.tank_detector.load_state_dict(weights['tank_detector'])
        self.shadow_analyzer.load_state_dict(weights['shadow_analyzer'])
        self.volume_estimator.load_state_dict(weights['volume_estimator'])

    def calculate_sun_position(self, timestamp):
        """Calculate sun's altitude and azimuth for shadow analysis"""
        if timestamp.tzinfo is None:
            utc_time = Time(utc_time, scale='utc')
        else:
            utc_time = Time(timestamp.astimezone(pytz.UTC), scale='utc')

        altaz = AltAz(location=self.location, obstime=utc_time)
        sun = get_sun(utc_time)
        sun_altaz = sun.transform_to(altaz)
        return sun_altaz.alt.deg, sun_altaz.az.deg

    def detect_tanks(self, image):
        """Detect tanks using the neural network"""
        self.tank_detector.eval()
        with torch.no_grad():
            img_tensor = self.transform(image).unsqueeze(0).to(self.device)
            locations, confidence = self.tank_detector(img_tensor)

            # Filter detections by confidence
            valid_detections = confidence.squeeze() > 0.5
            locations = locations[valid_detections]

            return locations.cpu().numpy()

    def analyze_shadow(self, image, tank_location, sun_alt, sun_az):
        """Analyze shadows using the shadow analysis network"""
        self.shadow_analyzer.eval()
        with torch.no_grad():
            # Extract tank region
            x, y, w, h = tank_location
            tank_region = image[int(y):int(y+h), int(x):int(x+w)]

            # Preprocess and get shadow mask
            tank_tensor = self.transform(tank_region).unsqueeze(0).to(self.device)
            shadow_mask = self.shadow_analyzer(tank_tensor)

            # Calculate shadow metrics
            shadow_area = shadow_mask.sum().item()
            shadow_perimeter = self.calculate_shadow_perimeter(shadow_mask)
            shadow_direction = self.calculate_shadow_direction(shadow_mask)

            return shadow_area, shadow_perimeter, shadow_direction

    @staticmethod
    def calculate_shadow_perimeter(shadow_mask):
        """Calculate the perimeter of the shadow"""
        shadow_np = shadow_mask.cpu().numpy().squeeze()
        edges = cv2.Canny(shadow_np.astype(np.uint8), 100, 200)
        return np.sum(edges > 0)

    @staticmethod
    def calculate_shadow_direction(shadow_mask):
        """Calculate the primary direction of the shadow"""
        shadow_np = shadow_mask.cpu().numpy().squeeze()
        moments = cv2.moments(shadow_np.astype(np.uint8))
        if moments['mu20'] + moments['mu02'] != 0:
            theta = 0.5 * np.arctan2(2 * moments['mu11'],
                                   moments['mu20'] - moments['mu02'])
            return np.degrees(theta)
        return 0

    def estimate_volume(self, shadow_metrics, tank_dims, sun_position):
        """Estimate tank volume using the volume estimation network"""
        self.volume_estimator.eval()
        with torch.no_grad():
            # Combine all features
            features = np.concatenate([
                shadow_metrics,  # shadow area, perimeter, direction
                tank_dims,       # width, height
                sun_position     # altitude, azimuth
            ])

            # Normalize features
            features_normalized = self.scaler.transform(features.reshape(1, -1))
            features_tensor = torch.FloatTensor(features_normalized).to(self.device)

            # Predict fill percentage
            fill_percentage = self.volume_estimator(features_tensor).item()

            # Calculate volume in barrels
            tank_radius = tank_dims[0] / 2
            height_meters = 14  # standard tank height
            total_volume = np.pi * (tank_radius ** 2) * height_meters
            volume_barrels = total_volume * 6.2898 * fill_percentage

            return fill_percentage * 100, volume_barrels

    def analyze_image(self, image_path, timestamp):
        """Main function to analyze satellite image and estimate storage volumes"""
        # Load and preprocess image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Could not load image")

        # Get sun position
        sun_alt, sun_az = self.calculate_sun_position(timestamp)

        # Detect tanks
        tank_locations = self.detect_tanks(image)

        results = []
        for location in tank_locations:
            # Analyze shadow
            shadow_metrics = self.analyze_shadow(image, location, sun_alt, sun_az)

            # Estimate volume
            tank_dims = location[2:]  # width, height
            sun_position = np.array([sun_alt, sun_az])
            fill_percentage, volume = self.estimate_volume(
                np.array(shadow_metrics),
                tank_dims,
                sun_position
            )

            results.append({
                'x': location[0],
                'y': location[1],
                'width': location[2],
                'height': location[3],
                'fill_percentage': fill_percentage,
                'estimated_volume': volume
            })

        return pd.DataFrame(results)

def main():
    analyzer = MLTankAnalyzer()
    # Get current time in UTC
    timestamp = datetime.now(pytz.UTC)
    image_path = '/content/gas.jpg'

    try:
        results = analyzer.analyze_image(image_path, timestamp)
        central_time = timestamp.astimezone(pytz.timezone('America/Chicago'))
        print(f"Analysis timestamp (Central Time): {central_time}")
        print(f"Total estimated storage: {results['estimated_volume'].sum():,.0f} barrels")
        print("\nIndividual tank details:")
        print(results.to_string(index=False))
        return results
    except Exception as e:
        print(f"Analysis failed: {str(e)}")
        return None

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are depreca

Analysis timestamp (Central Time): 2024-12-29 02:22:51.847909-06:00
Analysis failed: 'estimated_volume'


In [26]:
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
import pytz
from astropy.coordinates import get_sun, EarthLocation, AltAz
from astropy.time import Time
import astropy.units as u
import pandas as pd
from torchvision import transforms, models
from sklearn.preprocessing import StandardScaler

class TankDetectionNet(nn.Module):
    def __init__(self):
        super(TankDetectionNet, self).__init__()
        # Use ResNet50 as backbone with updated weights parameter
        self.backbone = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

        # Modify final layers for tank detection
        num_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()

        # Custom detection heads
        self.location_head = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 4)  # x, y, width, height
        )

        self.confidence_head = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        features = self.backbone(x)
        locations = self.location_head(features)
        confidence = self.confidence_head(features)
        return locations, confidence

class ShadowAnalysisNet(nn.Module):
    def __init__(self):
        super(ShadowAnalysisNet, self).__init__()
        # U-Net style architecture for shadow segmentation
        self.encoder = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        self.encoder.fc = nn.Identity()

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2),
            nn.Sigmoid()
        )

    def forward(self, x):
        features = self.encoder(x)
        features = features.view(-1, 512, 7, 7)  # Reshape for decoder
        shadow_mask = self.decoder(features)
        return shadow_mask

class VolumeEstimationNet(nn.Module):
    def __init__(self):
        super(VolumeEstimationNet, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(7, 128),  # Input: shadow metrics + sun position + tank dimensions
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)    # Output: fill percentage
        )

    def forward(self, x):
        return self.network(x)

class MLTankAnalyzer:
    def __init__(self, model_weights_path=None):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Initialize models
        self.tank_detector = TankDetectionNet().to(self.device)
        self.shadow_analyzer = ShadowAnalysisNet().to(self.device)
        self.volume_estimator = VolumeEstimationNet().to(self.device)

        # Load pre-trained weights if provided
        if model_weights_path:
            self.load_weights(model_weights_path)

        # Cushing coordinates
        self.latitude = 35.9828  # N
        self.longitude = -96.7534  # W
        self.location = EarthLocation(lat=self.latitude*u.deg,
                                    lon=self.longitude*u.deg,
                                    height=300*u.m)

        # Image preprocessing
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
        ])

        self.scaler = StandardScaler()

    def load_weights(self, weights_path):
        """Load pre-trained weights for all models"""
        weights = torch.load(weights_path, map_location=self.device)
        self.tank_detector.load_state_dict(weights['tank_detector'])
        self.shadow_analyzer.load_state_dict(weights['shadow_analyzer'])
        self.volume_estimator.load_state_dict(weights['volume_estimator'])

    def calculate_sun_position(self, timestamp):
        """Calculate sun's altitude and azimuth for shadow analysis"""
        from astropy.time import Time

        # Convert datetime to Astropy Time object
        if timestamp.tzinfo is None:
            utc_time = pytz.UTC.localize(timestamp)
        else:
            utc_time = timestamp.astimezone(pytz.UTC)

        time = Time(utc_time)

        # Calculate sun position
        altaz = AltAz(location=self.location, obstime=time)
        sun = get_sun(time)
        sun_altaz = sun.transform_to(altaz)
        return sun_altaz.alt.deg, sun_altaz.az.deg

    def detect_tanks(self, image):
        """Detect tanks using the neural network"""
        self.tank_detector.eval()
        with torch.no_grad():
            img_tensor = self.transform(image).unsqueeze(0).to(self.device)
            locations, confidence = self.tank_detector(img_tensor)

            # Filter detections by confidence
            valid_detections = confidence.squeeze() > 0.5
            locations = locations[valid_detections]

            return locations.cpu().numpy()

    def analyze_shadow(self, image, tank_location, sun_alt, sun_az):
        """Analyze shadows using the shadow analysis network"""
        self.shadow_analyzer.eval()
        with torch.no_grad():
            # Extract tank region
            x, y, w, h = tank_location
            tank_region = image[int(y):int(y+h), int(x):int(x+w)]

            # Preprocess and get shadow mask
            tank_tensor = self.transform(tank_region).unsqueeze(0).to(self.device)
            shadow_mask = self.shadow_analyzer(tank_tensor)

            # Calculate shadow metrics
            shadow_area = shadow_mask.sum().item()
            shadow_perimeter = self.calculate_shadow_perimeter(shadow_mask)
            shadow_direction = self.calculate_shadow_direction(shadow_mask)

            return shadow_area, shadow_perimeter, shadow_direction

    @staticmethod
    def calculate_shadow_perimeter(shadow_mask):
        """Calculate the perimeter of the shadow"""
        shadow_np = shadow_mask.cpu().numpy().squeeze()
        edges = cv2.Canny(shadow_np.astype(np.uint8), 100, 200)
        return np.sum(edges > 0)

    @staticmethod
    def calculate_shadow_direction(shadow_mask):
        """Calculate the primary direction of the shadow"""
        shadow_np = shadow_mask.cpu().numpy().squeeze()
        moments = cv2.moments(shadow_np.astype(np.uint8))
        if moments['mu20'] + moments['mu02'] != 0:
            theta = 0.5 * np.arctan2(2 * moments['mu11'],
                                   moments['mu20'] - moments['mu02'])
            return np.degrees(theta)
        return 0

    def estimate_volume(self, shadow_metrics, tank_dims, sun_position):
        """Estimate tank volume using the volume estimation network"""
        self.volume_estimator.eval()
        with torch.no_grad():
            # Combine all features
            features = np.concatenate([
                shadow_metrics,  # shadow area, perimeter, direction
                tank_dims,       # width, height
                sun_position     # altitude, azimuth
            ])

            # Normalize features
            features_normalized = self.scaler.transform(features.reshape(1, -1))
            features_tensor = torch.FloatTensor(features_normalized).to(self.device)

            # Predict fill percentage
            fill_percentage = self.volume_estimator(features_tensor).item()

            # Calculate volume in barrels
            tank_radius = tank_dims[0] / 2
            height_meters = 14  # standard tank height
            total_volume = np.pi * (tank_radius ** 2) * height_meters
            volume_barrels = total_volume * 6.2898 * fill_percentage

            return fill_percentage * 100, volume_barrels

    def analyze_image(self, image_path, timestamp):
        """Main function to analyze satellite image and estimate storage volumes"""
        # Load and preprocess image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Could not load image")

        # Get sun position
        sun_alt, sun_az = self.calculate_sun_position(timestamp)

        # Detect tanks
        tank_locations = self.detect_tanks(image)

        if len(tank_locations) == 0:
            return pd.DataFrame(columns=['x', 'y', 'width', 'height',
                                      'fill_percentage', 'estimated_volume'])

        results = []
        for location in tank_locations:
            try:
                # Analyze shadow
                shadow_metrics = self.analyze_shadow(image, location, sun_alt, sun_az)

                # Estimate volume
                tank_dims = location[2:]  # width, height
                sun_position = np.array([sun_alt, sun_az])
                fill_percentage, volume = self.estimate_volume(
                    np.array(shadow_metrics),
                    tank_dims,
                    sun_position
                )

                results.append({
                    'x': float(location[0]),
                    'y': float(location[1]),
                    'width': float(location[2]),
                    'height': float(location[3]),
                    'fill_percentage': float(fill_percentage),
                    'estimated_volume': float(volume)
                })
            except Exception as e:
                print(f"Error processing tank at location {location}: {str(e)}")
                continue

        return pd.DataFrame(results)

def main():
    analyzer = MLTankAnalyzer()
    # Get current time in UTC
    timestamp = datetime.now(pytz.UTC)
    image_path = '/content/gas.jpg'

    try:
        results = analyzer.analyze_image(image_path, timestamp)
        central_time = timestamp.astimezone(pytz.timezone('America/Chicago'))
        print(f"Analysis timestamp (Central Time): {central_time}")
        print(f"Total estimated storage: {results['estimated_volume'].sum():,.0f} barrels")
        print("\nIndividual tank details:")
        print(results.to_string(index=False))
        return results
    except Exception as e:
        print(f"Analysis failed: {str(e)}")
        return None

if __name__ == "__main__":
    main()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 207MB/s]


Error processing tank at location [[ 0.15552461 -0.02386813  0.02027518  0.14059772]]: not enough values to unpack (expected 4, got 1)
Analysis timestamp (Central Time): 2024-12-29 02:28:07.759219-06:00
Analysis failed: 'estimated_volume'
